# Componente 2: Publicación de datos abiertos
Muchos datos de valor del estado paraguayo se encuentran publicados en formato subóptimo
de acuerdo a las buenas prácticas de datos abiertos, por lo cual su potencial se desperdicia al
no ser procesables por máquinas.
Un ejemplo de esto es la lista puntos de venta de billetaje electrónico disponible en
https://www.mopc.gov.py/billetaje-electronico/ la cual se nos presenta de manera interactiva y
visualmente atractiva, pero sin posibilidad de descarga en un formato abierto, ni mención de la
licencia de publicación.
El alumno deberá extraer los datos de un dataset (el cual la descarga no esté disponible
fácilmente) utilizando las técnicas de web Scraping con Python aprendidas en clase, para
finalmente almacenarlos en formato csv. Como entregable de este componente se espera un
repositorio github con:
* Código fuente
* README.md con instrucciones de instalación y uso del script
* Dataset en formato .csv
* Diccionario de datos
* Licencia de publicación

In [3]:
#importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [14]:
#Función de scrapear las paginas de celulares de comprasparaguai.com.br
nombres_celulares = []
precios_celulares = []
list_celulares = []

no_pages = 5    #Cantidad de paginas a scrapear
for i in range(1, no_pages+1):  
    #El ID que va usar el scraper
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"} 

    #Sacar todo el contenido de la página
    r = requests.get('https://www.comprasparaguai.com.br/celular/?page='+str(i), headers=headers)
    content = r.content
    soup = BeautifulSoup(content)

    

    #Ahora extraemos solamente los datos que deseamos
    for container_item in soup.find_all('div', attrs={'class':'promocao-produtos-item col-sm-12'}):     #Los contenedores generales de los celulares
        item = container_item.find('a', attrs={'class':'truncate'})                                     #El nombre del artículo
        precio = container_item.find('span')                                                            #El precio del artículo
        precios_celulares.append(precio.get_text(strip=True))
        nombres_celulares.append(item.get_text(strip=True))
        
    

In [15]:
#Guardamos los datos en un array de numpy
list_celulares = np.array([nombres_celulares, precios_celulares])

In [16]:
#transposamos el array porque lo guardó de forma invertida
list_celulares = np.transpose(list_celulares)

In [17]:
#imprimimos la lista para verificar el contenido
list_celulares

array([['Celular Apple iPhone 13 128GB', 'U$\xa0812,00'],
       ['Celular Apple iPhone 13 Pro Max 128GB', 'U$\xa01.110,00'],
       ['Celular Apple iPhone 13 Pro 128GB', 'U$\xa01.084,00'],
       ['Celular Samsung Galaxy S22+ SM-S906E Dual Chip 256GB 5G',
        'U$\xa0900,00'],
       ['Celular Samsung Galaxy S22 SM-S901E Dual Chip 256GB 5G',
        'U$\xa0780,00'],
       ['Celular Samsung Galaxy S22 Ultra SM-S908E Dual Chip 256GB 5G',
        'U$\xa01.110,00'],
       ['Celular Apple iPhone XR 64GB Recondicionado', 'U$\xa0245,00'],
       ['Celular Apple iPhone XR 128GB Recondicionado', 'U$\xa0290,00'],
       ['Celular Xiaomi Redmi 9A Dual Chip 32GB 4G Global',
        'U$\xa091,75'],
       ['Celular Apple iPhone 8 Plus 64GB Recondicionado',
        'U$\xa0220,00'],
       ['Celular Apple iPhone 13 256GB', 'U$\xa0879,00'],
       ['Celular Apple iPhone 11 64GB', 'U$\xa0505,00'],
       ['Celular Xiaomi Redmi 9C Dual Chip 64GB 4G Global',
        'U$\xa0115,50'],
       ['Celula

In [18]:
#Convertimos el array de numpy en un Dataframe
data = pd.DataFrame(list_celulares,columns=['Celulares','Precios'])

In [19]:
#funcion para limpiar los precios
import re
def cleanPrice(text):
    text = re.sub("[.]", '', text)            # Sacando los '.'
    text = re.sub(',', '.', text)             # Reemplazando las comas restantes con puntos
    text = re.sub('[U$Â ]', '', text)             # Sacando los signos "$""
    return text

#Funcion para limpieza básica de los nombres de los artículos
def cleanTxt(text):
    text = re.sub('Celular ', '', text)             # Sacando la palabra Celulares
 
    return text

In [20]:
#Aplicamos los métodos de limpieza
data['Precios'] = data['Precios'].apply(cleanPrice)
data['Celulares'] = data['Celulares'].apply(cleanTxt)

In [21]:
#Convertir los precios de string a numerico
data['Precios'] = pd.to_numeric(data['Precios'])

In [22]:
#Imprimos los resultados
data

,Celulares,Precios
0,Apple iPhone 13 128GB,812.0
1,Apple iPhone 13 Pro Max 128GB,1110.0
2,Apple iPhone 13 Pro 128GB,1084.0
3,Samsung Galaxy S22+ SM-S906E Dual Chip 256GB 5G,900.0
4,Samsung Galaxy S22 SM-S901E Dual Chip 256GB 5G,780.0
...,...,...
95,Apple iPhone 13 Pro Max 512GB,1545.0
96,Xiaomi Redmi Note 10S Dual Chip 128GB 4G - RAM...,195.0
97,Blu Z4 Z194 Dual Chip,11.9
98,Realme C21Y RMX3261 Dual Chip 64GB 4G,112.5


In [23]:
#Exportar los datos a un archivo CSV limpiado
data.to_csv('comprasparaguai_celulares.csv')